In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization
import tensorflow as tf
import logging
from typing import Dict, List, Tuple
import numpy as np
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import History

In [2]:
class NeuralNetwork:
    """
    Class for deploying neural network
    """
        
    def model_prepare(self, merge_models: tf.Tensor, inputs: List[tf.Tensor]) -> Model:
        """
        Creates a multi-input and multi-output neural network model 

        Args:
            merge_models: A tensor that combined continuous features and embeddings from categorical features
            inputs: A list containing the Input layer for each feature

        Returns:
            Model: A prepared structure of model 
        """
        try:
            x = Dense(100, activation='relu')(merge_models)
            x = BatchNormalization()(x)
            x = Dense(100, activation='relu')(x)
            x = BatchNormalization()(x)

            home_score_output = Dense(1, name="home_score")(x)
            away_score_output = Dense(1, name="away_score")(x)

            model = Model(inputs=inputs, outputs=[home_score_output, away_score_output])
            optimizer = SGD(learning_rate=0.001)
            model.compile(optimizer=optimizer, 
                          loss={"home_score": "mse", "away_score": "mse"},
                          metrics={"home_score": RootMeanSquaredError(),
                                   "away_score": RootMeanSquaredError()                                  
                                   })
            model.summary()
            return model
        except Exception as e:
            logging.error(f"Error in model preparation")
            raise e


    def model_train(self, 
                    model: Model, 
                    training_dataset: Dict[str, Dict[str, np.ndarray]], 
                    validation_dataset: Dict[str, Dict[str, np.ndarray]], 
                    epochs: int = 10
                    ) -> Tuple[Model, History]:
        """
        Trains the model on the training dataset and validate on new dataset

        Args:
            model: Allready prepared model
            training_dataset: Training dataset
            validation_dataset: Validation dataset
            epochs: Number of epochs for model to train
        Returns:
            Model: Trained model
            history: History of metrics all epochs during training
        """
        try:
            training_features = training_dataset["input_features"]
            training_labels = training_dataset["labels"]
            validation_features = validation_dataset["input_features"]
            validation_labels = validation_dataset["labels"]
            history = model.fit(training_features, 
                      training_labels, 
                      epochs=epochs,
                      batch_size=100,
                      validation_data=(validation_features, 
                                       validation_labels))
            return model, history
        except Exception as e:
            logging.error(f"Error in model training: {e}")
            raise e